# Profitable App Profiles on App Store and Google Play
by Nicholas Archambault

Between the Apple Store and Google Play, over 4 million apps are available for download.  This project seeks to understand which types of free apps are most likely to attract and maintain users.

As of September 2018, there were approximately 2 million iOS apps available on App Store and 2.1 million Android apps on Google Play.

Since collection and analysis of so much data would take time and money, we will analyze a sample of this data to determine for developers which apps should be targeted for revenue-producing advertisement. 

We start by reading in and opening the datasets for the App Store and Google Play.

In [1]:
from csv import reader
apple = open("AppleStore.csv")
google = open("googleplaystore.csv")

apple = reader(apple)
google = reader(google)

apple = list(apple)
header_apple = apple[0]
apple = apple[1:]

google = list(google)
header_google = google[0]
google = google[1:]

To make it easier to explore the two data sets, we'll first write a function named `explore_data()` that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # Adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
# Rows
print(len(apple))
print(len(google))

# Columns
print(len(apple[0]))
print(len(google[0]))

7197
10841
16
13


At a glance, we can see that the Apple dataset has dimensions of 7,197 rows by 16 columns, and that the Google dataset has dimensions of 10,841 rows by 13 columns.

## Deleting Incorrect Data
The row 10,472 in the Google dataset corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5. As such, we'll delete this row.

In [4]:
del google[10472]

## Removing Duplicate Entries
### Part One
If we explore the Google Play data set long enough, we'll find that some apps have more than one entry. In total, there are 1,181 cases where an app occurs more than once, and 9,659 unique apps.

In [5]:
duplicate_apps = []
unique_apps = []
for i in google:
    name = i[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print("Duplicate apps: {}".format(len(duplicate_apps)))
print("Unique apps: {}".format(len(unique_apps)))

Duplicate apps: 1181
Unique apps: 9659


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app.

The main difference between duplicate apps occurs on the fourth position of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. We can use this to build a criterion for keeping rows. We won't remove rows randomly, but rather we'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

### Part Two
To remove duplicate rows, we can first initialize a dictionary showing the name and highest number of reviews for each unique app. 

In [6]:
reviews_max = {}
for i in google:
    name = i[0]
    n_reviews = float(i[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
len(reviews_max)

9659

Next, we'll initialize two lists. For every app in the data set, we'll add the row to `google_clean` and the name to the `already_added` if the number of reviews matches the maximum for that app and if the app has not already been added. 

In [7]:
google_clean = []
already_added = []
for i in google:
    name = i[0]
    n_reviews = float(i[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        google_clean.append(i)
        already_added.append(name)
len(google_clean)

9659

We find a dictionary length of 9,659, as expected.

## Removing Non-English Apps
Some app names are not in English, indicating that the apps do not target English-speaking audiences.  We'll remove those apps.

We have determined that the apps to be eliminated include all those with more than three non-English text characters based on ASCII classification. We can create a function that will determine whether an app should be removed. 

In [8]:
def language(string):
    num = 0
    for i in string:
        if ord(i) > 127:
            num += 1
    if num > 3:
        return False
    return True

In [9]:
# Test cases
print(language("Instagram"))
print(language("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(language("Docs To Go™ Free Office Suite"))
print(language("Instachat 😜"))

True
False
True
True


The boolean mask that's been created can then be applied to both datasets to remove all non-English apps.

In [10]:
english_apps_google = []
for i in google_clean:
    choice = language(i[0])
    if choice == True:
        english_apps_google.append(i)
print(len(english_apps_google))

9614


In [11]:
english_apps_apple = []
for i in apple:
    choice = language(i[1])
    if choice == True:
        english_apps_apple.append(i)
print(len(english_apps_apple))

6183


We're left with 9,614 Android apps and 6,183 Apple apps.

## Isolating Free Apps

To continue our analysis of which apps ought to be targeted for revenue-producing advertisements, we must filter the datasets to include only free apps.

In [12]:
free_apple = []
for i in english_apps_apple:
    if i[4] == '0.0':
        free_apple.append(i)
print(len(free_apple))

3222


In [13]:
free_google = []
for i in english_apps_google:
    if i[7] == '0':
        free_google.append(i)
print(len(free_google))

8864


The datasets now contain 3,222 Apple apps and 8,864 Android apps.

## Most Common Apps by Genre

As previously mentioned, our aim is to understand which types of apps are most likely to attract users in order to target advertisements appropriately. 

To minimize risks and overhead, the validation strategy for an app idea is comprised of three steps:

   1. Build a minimal Android version of the app, and add it to Google Play.
   2. If the app has a good response from users, we then develop it further.
   3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

We can begin our analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the `prime_genre` column of the App Store data set, and the `Genres` and `Category` columns of the Google Play data set.

In [14]:
# Generate frequency tables that show percentages.
def freq_table(dataset, index):
    mydict = {}
    for i in dataset:
        column = i[index]
        
        if column in mydict:
            mydict[column] += 1
        else:
            mydict[column] = 1
    
    for j in mydict:
        mydict[j] /= len(dataset)
        mydict[j] *= 100
        mydict[j] = round(mydict[j], 2)
    
    return mydict

# Display percentages in descending order.
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

### Apple Apps

In [15]:
display_table(free_apple, 11) # `prime_genre` column of Apple dataset

Games : 58.16
Entertainment : 7.88
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.51
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.33
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps, which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. 

### Android Apps

In [16]:
display_table(free_google, 9) # Category column

Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.91
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;Action & Adventure : 0.17
Puzzle;Brain Games : 0.17
Entertainment;Music & Video : 0.17
Casual;

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). The popularity of practical apps is confirmed by the frequency table we see for the Genres column:

In [17]:
display_table(free_google, 1) # Genre column

FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6


The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular. We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on App Store

One way to find out what genres are the most popular is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the `Installs` column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the `rating_count_tot` column.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [18]:
# Define list of genres
genre_list = []
for i in free_apple:
    genre = i[11]
    if genre not in genre_list:
        genre_list.append(genre)

print(genre_list)   

['Social Networking', 'Photo & Video', 'Games', 'Music', 'Reference', 'Health & Fitness', 'Weather', 'Utilities', 'Travel', 'Shopping', 'News', 'Navigation', 'Lifestyle', 'Entertainment', 'Food & Drink', 'Sports', 'Book', 'Finance', 'Education', 'Productivity', 'Business', 'Catalogs', 'Medical']


In [19]:
# Increment counts for each genre
for genre in genre_list:
    total = 0
    len_genre = 0
    
    for i in free_apple:
        genre_app = i[11]
        
        if genre_app == genre:
            ratings = float(i[5])
            total += ratings
            len_genre += 1
    avg_ratings = total/len_genre
    print(genre, ":", round(int(avg_ratings), 0))

Social Networking : 71548
Photo & Video : 28441
Games : 22788
Music : 57326
Reference : 74942
Health & Fitness : 23298
Weather : 52279
Utilities : 18684
Travel : 28243
Shopping : 26919
News : 21248
Navigation : 86090
Lifestyle : 16485
Entertainment : 14029
Food & Drink : 33333
Sports : 23008
Book : 39758
Finance : 31467
Education : 7003
Productivity : 21028
Business : 7491
Catalogs : 4004
Medical : 612


On average, navigation, social networking, and music apps garner the highest number of user reviews.  These results, however, are skewed by the top apps that dominate each category, including Waze and Google Maps for navigation; Facebook, Pinterest and Skype for social networking; and Pandora, Spotify, and Shazam for music.

In [20]:
# Define list of genres
for app in free_apple:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


In [21]:
# Increment count for each genre
for app in free_apple:
    if app[-5] == 'Social Networking':
        print(app[1], ':', app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

In [22]:
# Check listings of Music apps and find that it is dominated by a few apps, especially Pandora and Spotify
for app in free_apple:
    if app[-5] == 'Music':
        print(app[1], ':', app[5])

Pandora - Music & Radio : 1126879
Spotify Music : 878563
Shazam - Discover music, artists, videos & lyrics : 402925
iHeartRadio – Free Music & Radio Stations : 293228
SoundCloud - Music & Audio : 135744
Magic Piano by Smule : 131695
Smule Sing! : 119316
TuneIn Radio - MLB NBA Audiobooks Podcasts Music : 110420
Amazon Music : 106235
SoundHound Song Search & Music Player : 82602
Sonos Controller : 48905
Bandsintown Concerts : 30845
Karaoke - Sing Karaoke, Unlimited Songs! : 28606
My Mixtapez Music : 26286
Sing Karaoke Songs Unlimited with StarMaker : 26227
Ringtones for iPhone & Ringtone Maker : 25403
Musi - Unlimited Music For YouTube : 25193
AutoRap by Smule : 18202
Spinrilla - Mixtapes For Free : 15053
Napster - Top Music & Radio : 14268
edjing Mix:DJ turntable to remix and scratch music : 13580
Free Music - MP3 Streamer & Playlist Manager Pro : 13443
Free Piano app by Yokee : 13016
Google Play Music : 10118
Certified Mixtapes - Hip Hop Albums & Mixtapes : 9975
TIDAL : 7398
YouTube Mu

Other popular genres include reference, weather, and food & drink. The reference genre is dominated by the Bible and dictionary apps, but holds promise for our app development goals.  Based on other high-performing apps in that category, it seems feasible and potentially successful to create an app version of a popular book, accompanied by additional materials besides the raw book text. Such materials could include audio quizzes, daily quotes, or authorial interviews. 

Since the market appears to be dominated by fun-focused apps, a more practical app like this may have the opportunity to stand out.

## Most Popular Apps by Genre on Google Play

In [23]:
# Understand app installation popularity on Google Play
display_table(free_google, 5)

1,000,000+ : 15.73
100,000+ : 11.55
10,000,000+ : 10.55
10,000+ : 10.2
1,000+ : 8.39
100+ : 6.92
5,000,000+ : 6.83
500,000+ : 5.56
50,000+ : 4.77
5,000+ : 4.51
10+ : 3.54
500+ : 3.25
50,000,000+ : 2.3
100,000,000+ : 2.13
50+ : 1.92
5+ : 0.79
1+ : 0.51
500,000,000+ : 0.27
1,000,000,000+ : 0.23
0+ : 0.05
0 : 0.01


A problem with this data is that the number of installs is not precise enough. To extract more meaningful data, we'll convert all these figures to floats.

In [24]:
# Define list of categories
categories = []
for i in free_google:
    category = i[1]
    if category not in categories:
        categories.append(category)

print(categories)

['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE', 'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL', 'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL', 'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER', 'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION']


In [25]:
# Increment counts for each category
for category in categories:
    total = 0
    len_category = 0
    
    for i in free_google:
        category_app = i[1]
        
        if category_app == category:
            installs = i[5]
            installs = installs.replace("+", "")
            installs = installs.replace(",", "")
            installs = float(installs)
            total += installs
            len_category += 1
    
    avg_installs = total/len_category
    print(category, ":", round(int(avg_installs), 0))

ART_AND_DESIGN : 1986335
AUTO_AND_VEHICLES : 647317
BEAUTY : 513151
BOOKS_AND_REFERENCE : 8767811
BUSINESS : 1712290
COMICS : 817657
COMMUNICATION : 38456119
DATING : 854028
EDUCATION : 1833495
ENTERTAINMENT : 11640705
EVENTS : 253542
FINANCE : 1387692
FOOD_AND_DRINK : 1924897
HEALTH_AND_FITNESS : 4188821
HOUSE_AND_HOME : 1331540
LIBRARIES_AND_DEMO : 638503
LIFESTYLE : 1437816
GAME : 15588015
FAMILY : 3695641
MEDICAL : 120550
SOCIAL : 23253652
SHOPPING : 7036877
PHOTOGRAPHY : 17840110
SPORTS : 3638640
TRAVEL_AND_LOCAL : 13984077
TOOLS : 10801391
PERSONALIZATION : 5201482
PRODUCTIVITY : 16787331
PARENTING : 542603
WEATHER : 5074486
VIDEO_PLAYERS : 24727872
NEWS_AND_MAGAZINES : 9549178
MAPS_AND_NAVIGATION : 4056941


On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [26]:
# View most-installed apps
for app in free_google:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

We find the same pattern holds for video, social, photography, and productivity apps, the next four most popular genres. Each is dominated and has its data skewed higher by the presence of a few apps that have been installed millions or billions of times.

It would be hard for a newly developed app to compete agains the established giants that rule these categories. And as with the Apple store, the game market seems nearly saturated with options.

With an average of 8,767,811 installs, the books and reference genre seems to be worth exploring, as it was on the Apple store. 

In [27]:
for app in free_google:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [28]:
for app in free_google:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential.

## Conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.